# CSV Databases Merger
By Stephen Karl Larroque @ Coma Science Group, GIGA Research, University of Liege
Creation date: 2018-05-27
License: MIT
v1.4.7
2018-2019

DESCRIPTION:
Generic tool to merge two CSV databases based on the subject's name (hence expecting a column 'name' in each csv file).
This script will take care of fuzzy matching names and append all columns of each csv file, hence centralizing all informations into one file.

Note: in case of multiple names/lines matching, they will all be concatenated into a single line: if one name in one of the two databases match multiple names/lines of the second database, then the second databases lines will be concatenated into one. In the opposite case (second database's name match multiple names in first database), the same holds.

Note2: please only run one of the PARAMETERS cells at a time (this selects appropriate parameters for the merge), and please follow the order.

INSTALL NOTE:
You need to pip install pandas before launching this script.
Tested on Python 2.7.15

USAGE:

TODO:
* Nothing here!

In [ ]:
# Forcefully autoreload all python modules
%load_ext autoreload
%autoreload 2

In [ ]:
# AUX FUNCTIONS

import os, sys

cur_path = os.path.realpath('.')
sys.path.append(os.path.join(cur_path, 'csg_fileutil_libs'))  # for unidecode and cleanup_name, because it does not support relative paths (yet?)

import re

from collections import OrderedDict
from csg_fileutil_libs.aux_funcs import save_df_as_csv, _tqdm, merge_two_df, df_remap_names, concat_vals, df_literal_eval, df_to_unicode, df_to_unicode_fast, reorder_cols_df, df_encode


In [ ]:
# PARAMETERS

# First (ID) database to merge (both need to have a column 'name'). The merged 'name' column will use the names from this database.
id_db = r'databases_original\patients-sedation-2018-checked-by-Stephen-from-archives_v7_2018-10-18_full-names-merged.csv'
id_db_tounicode = True  # convert at loading the database to unicode? (If you get issues with accentuated characters or special characters in the final merged database, try to set this to True)
# Second (reference) database to merge. The names will be added as a new column 'name_altx'.
ref_db = r'databases_output\fmp_db_subjects_aggregated.csv_etiosedatfixed_dicomsdatediag_dicompathsedat.csv_acute.csv'
ref_db_tounicode = True  # convert at loading the database to unicode? (If you get issues with accentuated characters or special characters in the final merged database, try to set this to True)
# Output database with the merge results
out_db = r'databases_output\merged_fmp_steph.csv'

# Similarity search parameters (of names in both databases)
dist_threshold = 0.2 # character distance (normalized on 1 over the total number of characters = jaccard distance), lower is more similar, default: 0.2
dist_words_threshold = 0.4 # words distance (normalized idem but on number of words), default: 0.2
keep_lastname_only = False # keep only the lastname? (supposed to be the first word) - this can enhance the matching if there are too many false positives, particularly if one database only include the last name but the other one has the full name

# Additional options
rename_columns_per_csv = ['steph.', 'fmpagg.']  # rename each column by prepending the csv file from where the column comes from. True will use the first 3 letters from the filename, or a list of 2 string prefixes can be provided, or False to disable renaming
#rename_columns_per_csv = False
pdmerge_indicator = False  # pandas.merge() option, if True, a new column 'x_merge' will be created for every columns, summarizing from what original database the info was merged from
pdmerge_validate = None  # pandas.merge() option, allows to ensure that the mapping is unique, for example with '1:1'. Can be: '1:1', '1:m', 'm:1', 'm:m'
blankna = False  # blank null values when saving to csv? This improves readability but the dates and numbers can't be reformatted then.
columns_order_first = None  # put these columns first when saving in a csv file. If None, no reordering will be done (except that the key columns will be first). Reordering columns (by cut/paste) might lead to losing some columns, so it is better to do it here programmatically.

# Multi-columns merging
# keys to use for the merge: by default will only use the 'name' column, but can merge on multiple columns and with different types: 'id' or 'datetime' are supported
# the order of the columns must be the same, since we will not use the column names: the id's database first column must match the same type as the ref's database first column, etc for all subsequent columns
# format is: an OrderedDict with the column_name: column_type for each item
#id_db_keys = None
#ref_db_keys = None
id_db_keys = OrderedDict([('name', 'id'), ('MRI date (on mri paper)', 'datetime|%d/%m/%Y')])
ref_db_keys = OrderedDict([('name', 'id'), ('Dicom Date Sync With CRS-R', 'datetime|%Y-%m-%d')])
shared_key_column = 'name'  # in case you set ref_db_keys and id_db_keys to None (to disable multi-columns merging), you can specify here the name of the single column that both databases share, and on which the merge will be done
fillna = True  # if merging on multiple key columns, if fillna=True this will try to fill nan fields from other filled fields from the same subject, eg: subject A has 2 sessions, one with lots of infos, second session mostly empty, with fillna=True the second session will get all missing infos copied over from the first (except for the key columns of course)
fillna_exclude = None  # allow to exclude some columns from being filled when fillna=True, this is useful particularly for datetime columns
join_on_shared_keys = False  # allow the merge on multi-columns to join on only the columns that are shared, this is useful when you want to groupby differently between dataframes (eg, first dataframe with name and date, the second with name only) but you want to keep all rows (else if you join on only name for example, all date rows will be concatenated)


In [ ]:
# PARAMETERS FOR 2ND MERGE (skip this cell if you did not do the first merge beforehand)

# First (ID) database to merge (both need to have a column 'name'). The merged 'name' column will use the names from this database.
id_db = r'databases_output\merged_fmp_steph.csv'
id_db_tounicode = True  # convert at loading the database to unicode? (If you get issues with accentuated characters or special characters in the final merged database, try to set this to True)
# Second (reference) database to merge. The names will be added as a new column 'name_altx'.
ref_db = r'databases_original\manon_Database_MRI_patients.csv'
ref_db_tounicode = True  # convert at loading the database to unicode? (If you get issues with accentuated characters or special characters in the final merged database, try to set this to True)
# Output database with the merge results
out_db = r'databases_output\merged_fmp_steph_manon.csv'

# Similarity search parameters (of names in both databases)
dist_threshold = 0.2 # character distance (normalized on 1 over the total number of characters = jaccard distance), lower is more similar, default: 0.2
dist_words_threshold = 0.4 # words distance (normalized idem but on number of words), default: 0.2
keep_lastname_only = False # keep only the lastname? (supposed to be the first word) - this can enhance the matching if there are too many false positives, particularly if one database only include the last name but the other one has the full name

# Additional options
rename_columns_per_csv = ['', 'manon.'] # rename each column by prepending the csv file from where the column comes from. True will use the first 3 letters from the filename, or a list of 2 string prefixes can be provided, or False to disable renaming
#rename_columns_per_csv = False
pdmerge_indicator = False  # pandas.merge() option, if True, a new column 'x_merge' will be created for every columns, summarizing from what original database the info was merged from
pdmerge_validate = None  # pandas.merge() option, allows to ensure that the mapping is unique, for example with '1:1'. Can be: '1:1', '1:m', 'm:1', 'm:m'
blankna = False  # blank null values when saving to csv? This improves readability but the dates and numbers can't be reformatted then.
columns_order_first = None  # put these columns first. If None, no reordering will be done (except that the key columns will be first).

# Multi-columns merging
# keys to use for the merge: by default will only use the 'name' column, but can merge on multiple columns and with different types: 'id' or 'datetime' are supported
# the order of the columns must be the same, since we will not use the column names: the id's database first column must match the same type as the ref's database first column, etc for all subsequent columns
# format is: an OrderedDict with the column_name: column_type for each item
id_db_keys = OrderedDict([('name', 'id'), ('MRI date (on mri paper) + Dicom Date Sync With CRS-R', 'datetime|%Y-%m-%d')])
ref_db_keys = OrderedDict([('name', 'id'), ('Date of RMN', 'datetime|%d/%m/%Y')])
shared_key_column = 'name'  # in case you set ref_db_keys and id_db_keys to None (to disable multi-columns merging), you can specify here the name of the single column that both databases share, and on which the merge will be done
fillna = True  # if merging on multiple key columns, if fillna=True this will try to fill nan fields from other filled fields from the same subject, eg: subject A has 2 sessions, one with lots of infos, second session mostly empty, with fillna=True the second session will get all missing infos copied over from the first (except for the key columns of course)
fillna_exclude = ['MRI date (on mri paper)', 'Dicom Date Sync With CRS-R']  # allow to exclude some columns from being filled when fillna=True, this is useful particularly for datetime columns
join_on_shared_keys = False  # allow the merge on multi-columns to join on only the columns that are shared, this is useful when you want to groupby differently between dataframes (eg, first dataframe with name and date, the second with name only) but you want to keep all rows (else if you join on only name for example, all date rows will be concatenated)


In [ ]:
# PARAMETERS FOR 3RD MERGE (skip this cell if you did not do the first merge beforehand)

# First (ID) database to merge (both need to have a column 'name'). The merged 'name' column will use the names from this database.
id_db = r'databases_output\merged_fmp_steph_manon.csv'
id_db_tounicode = True  # convert at loading the database to unicode? (If you get issues with accentuated characters or special characters in the final merged database, try to set this to True)
# Second (reference) database to merge. The names will be added as a new column 'name_altx'.
ref_db = r'databases_output\SarahBDD_crsr_all.csv'
ref_db_tounicode = True  # convert at loading the database to unicode? (If you get issues with accentuated characters or special characters in the final merged database, try to set this to True)
# Output database with the merge results
out_db = r'databases_output\merged_fmp_steph_manon_sarah.csv'

# Similarity search parameters (of names in both databases)
dist_threshold = 0.2 # character distance (normalized on 1 over the total number of characters = jaccard distance), lower is more similar, default: 0.2
dist_words_threshold = 0.4 # words distance (normalized idem but on number of words), default: 0.2
keep_lastname_only = False # keep only the lastname? (supposed to be the first word) - this can enhance the matching if there are too many false positives, particularly if one database only include the last name but the other one has the full name

# Additional options
rename_columns_per_csv = ['', 'sarah.'] # rename each column by prepending the csv file from where the column comes from. True will use the first 3 letters from the filename, or a list of 2 string prefixes can be provided, or False to disable renaming
#rename_columns_per_csv = False
pdmerge_indicator = False  # pandas.merge() option, if True, a new column 'x_merge' will be created for every columns, summarizing from what original database the info was merged from
pdmerge_validate = None  # pandas.merge() option, allows to ensure that the mapping is unique, for example with '1:1'. Can be: '1:1', '1:m', 'm:1', 'm:m'
blankna = False  # blank null values when saving to csv? This improves readability but the dates and numbers can't be reformatted then.
columns_order_first = None  # put these columns first. If None, no reordering will be done (except that the key columns will be first).

# Multi-columns merging
# keys to use for the merge: by default will only use the 'name' column, but can merge on multiple columns and with different types: 'id' or 'datetime' are supported
# the order of the columns must be the same, since we will not use the column names: the id's database first column must match the same type as the ref's database first column, etc for all subsequent columns
# format is: an OrderedDict with the column_name: column_type for each item
id_db_keys = OrderedDict([('name', 'id'), ('MRI date (on mri paper) + Dicom Date Sync With CRS-R + Date of RMN', 'datetime|%Y-%m-%d')])
ref_db_keys = OrderedDict([('name', 'id'), ('date_assess', 'datetime|%Y-%m-%d')])
shared_key_column = 'name'  # in case you set ref_db_keys and id_db_keys to None (to disable multi-columns merging), you can specify here the name of the single column that both databases share, and on which the merge will be done
fillna = False  # if merging on multiple key columns, if fillna=True this will try to fill nan fields from other filled fields from the same subject, eg: subject A has 2 sessions, one with lots of infos, second session mostly empty, with fillna=True the second session will get all missing infos copied over from the first (except for the key columns of course)
fillna_exclude = ['MRI date (on mri paper)', 'Dicom Date Sync With CRS-R', 'Date of RMN']  # allow to exclude some columns from being filled when fillna=True, this is useful particularly for datetime columns
join_on_shared_keys = False  # allow the merge on multi-columns to join on only the columns that are shared, this is useful when you want to groupby differently between dataframes (eg, first dataframe with name and date, the second with name only) but you want to keep all rows (else if you join on only name for example, all date rows will be concatenated)


In [ ]:
# PARAMETERS FOR 4TH MERGE (skip this cell if you did not do the first merge beforehand)

# First (ID) database to merge (both need to have a column 'name'). The merged 'name' column will use the names from this database.
id_db = r'databases_output\merged_fmp_steph_manon_sarah.csv'
id_db_tounicode = True  # convert at loading the database to unicode? (If you get issues with accentuated characters or special characters in the final merged database, try to set this to True)
# Second (reference) database to merge. The names will be added as a new column 'name_altx'.
ref_db = r'databases_output\dicoms_db_infosmatch_reorg.csv'
ref_db_tounicode = True  # convert at loading the database to unicode? (If you get issues with accentuated characters or special characters in the final merged database, try to set this to True)
# Output database with the merge results
out_db = r'databases_output\merged_fmp_steph_manon_sarah_dicom.csv'

# Similarity search parameters (of names in both databases)
dist_threshold = 0.2 # character distance (normalized on 1 over the total number of characters = jaccard distance), lower is more similar, default: 0.2
dist_words_threshold = 0.4 # words distance (normalized idem but on number of words), default: 0.2
keep_lastname_only = False # keep only the lastname? (supposed to be the first word) - this can enhance the matching if there are too many false positives, particularly if one database only include the last name but the other one has the full name

# Additional options
rename_columns_per_csv = ['', 'dicom.'] # rename each column by prepending the csv file from where the column comes from. True will use the first 3 letters from the filename, or a list of 2 string prefixes can be provided, or False to disable renaming
#rename_columns_per_csv = False
pdmerge_indicator = False  # pandas.merge() option, if True, a new column 'x_merge' will be created for every columns, summarizing from what original database the info was merged from
pdmerge_validate = None  # pandas.merge() option, allows to ensure that the mapping is unique, for example with '1:1'. Can be: '1:1', '1:m', 'm:1', 'm:m'
blankna = False  # blank null values when saving to csv? This improves readability but the dates and numbers can't be reformatted then.
columns_order_first = None  # put these columns first. If None, no reordering will be done (except that the key columns will be first).

# Multi-columns merging
# keys to use for the merge: by default will only use the 'name' column, but can merge on multiple columns and with different types: 'id' or 'datetime' are supported
# the order of the columns must be the same, since we will not use the column names: the id's database first column must match the same type as the ref's database first column, etc for all subsequent columns
# format is: an OrderedDict with the column_name: column_type for each item
id_db_keys = OrderedDict([('name', 'id'), ('MRI date (on mri paper) + Dicom Date Sync With CRS-R + Date of RMN + date_assess', 'datetime|%Y-%m-%d')])
ref_db_keys = OrderedDict([('name', 'id'), ('StudyDate', 'datetime|{%Y%m%d}')])
shared_key_column = 'name'  # in case you set ref_db_keys and id_db_keys to None (to disable multi-columns merging), you can specify here the name of the single column that both databases share, and on which the merge will be done
fillna = False  # if merging on multiple key columns, if fillna=True this will try to fill nan fields from other filled fields from the same subject, eg: subject A has 2 sessions, one with lots of infos, second session mostly empty, with fillna=True the second session will get all missing infos copied over from the first (except for the key columns of course)
fillna_exclude = ['MRI date (on mri paper)', 'Dicom Date Sync With CRS-R', 'Date of RMN', 'date_assess']  # allow to exclude some columns from being filled when fillna=True, this is useful particularly for datetime columns
join_on_shared_keys = False  # allow the merge on multi-columns to join on only the columns that are shared, this is useful when you want to groupby differently between dataframes (eg, first dataframe with name and date, the second with name only) but you want to keep all rows (else if you join on only name for example, all date rows will be concatenated)


In [ ]:
# PARAMETERS FOR 5TH MERGE (skip this cell if you did not do the first merge beforehand)

# First (ID) database to merge (both need to have a column 'name'). The merged 'name' column will use the names from this database.
id_db = r'databases_output\merged_fmp_steph_manon_sarah_dicom.csv'
id_db_tounicode = True  # convert at loading the database to unicode? (If you get issues with accentuated characters or special characters in the final merged database, try to set this to True)
# Second (reference) database to merge. The names will be added as a new column 'name_altx'.
ref_db = r'databases_output\ecg_subjects.csv'
ref_db_tounicode = True  # convert at loading the database to unicode? (If you get issues with accentuated characters or special characters in the final merged database, try to set this to True)
# Output database with the merge results
out_db = r'databases_output\merged_fmp_steph_manon_sarah_dicom_ecg.csv'

# Similarity search parameters (of names in both databases)
dist_threshold = 0.2 # character distance (normalized on 1 over the total number of characters = jaccard distance), lower is more similar, default: 0.2
dist_words_threshold = 0.4 # words distance (normalized idem but on number of words), default: 0.2
keep_lastname_only = False # keep only the lastname? (supposed to be the first word) - this can enhance the matching if there are too many false positives, particularly if one database only include the last name but the other one has the full name

# Additional options
rename_columns_per_csv = ['', 'ecg.'] # rename each column by prepending the csv file from where the column comes from. True will use the first 3 letters from the filename, or a list of 2 string prefixes can be provided, or False to disable renaming
#rename_columns_per_csv = False
pdmerge_indicator = False  # pandas.merge() option, if True, a new column 'x_merge' will be created for every columns, summarizing from what original database the info was merged from
pdmerge_validate = None  # pandas.merge() option, allows to ensure that the mapping is unique, for example with '1:1'. Can be: '1:1', '1:m', 'm:1', 'm:m'
blankna = False  # blank null values when saving to csv? This improves readability but the dates and numbers can't be reformatted then.
columns_order_first = None  # put these columns first. If None, no reordering will be done (except that the key columns will be first).

# Multi-columns merging
# keys to use for the merge: by default will only use the 'name' column, but can merge on multiple columns and with different types: 'id' or 'datetime' are supported
# the order of the columns must be the same, since we will not use the column names: the id's database first column must match the same type as the ref's database first column, etc for all subsequent columns
# format is: an OrderedDict with the column_name: column_type for each item
id_db_keys = OrderedDict([('name', 'id'), ('MRI date (on mri paper) + Dicom Date Sync With CRS-R + Date of RMN + date_assess + StudyDate', 'datetime|%Y-%m-%d')])
ref_db_keys = OrderedDict([('name', 'id')])
shared_key_column = 'name'  # in case you set ref_db_keys and id_db_keys to None (to disable multi-columns merging), you can specify here the name of the single column that both databases share, and on which the merge will be done
fillna = True  # if merging on multiple key columns, if fillna=True this will try to fill nan fields from other filled fields from the same subject, eg: subject A has 2 sessions, one with lots of infos, second session mostly empty, with fillna=True the second session will get all missing infos copied over from the first (except for the key columns of course)
fillna_exclude = ['MRI date (on mri paper)', 'Dicom Date Sync With CRS-R', 'Date of RMN', 'date_assess', 'StudyDate']  # allow to exclude some columns from being filled when fillna=True, this is useful particularly for datetime columns
join_on_shared_keys = True  # allow the merge on multi-columns to join on only the columns that are shared, this is useful when you want to groupby differently between dataframes (eg, first dataframe with name and date, the second with name only) but you want to keep all rows (else if you join on only name for example, all date rows will be concatenated)


In [ ]:
# PARAMETERS FOR 6TH MERGE (skip this cell if you did not do the first merge beforehand)

# First (ID) database to merge (both need to have a column 'name'). The merged 'name' column will use the names from this database.
id_db = r'databases_output\merged_fmp_steph_manon_sarah_dicom_ecg.csv'
id_db_tounicode = True  # convert at loading the database to unicode? (If you get issues with accentuated characters or special characters in the final merged database, try to set this to True)
# Second (reference) database to merge. The names will be added as a new column 'name_altx'.
ref_db = r'databases_output\all_patients_fields.csv'
ref_db_tounicode = True  # convert at loading the database to unicode? (If you get issues with accentuated characters or special characters in the final merged database, try to set this to True)
# Output database with the merge results
out_db = r'databases_output\merged_fmp_steph_manon_sarah_dicom_ecg_reports.csv'

# Similarity search parameters (of names in both databases)
dist_threshold = 0.2 # character distance (normalized on 1 over the total number of characters = jaccard distance), lower is more similar, default: 0.2
dist_words_threshold = 0.4 # words distance (normalized idem but on number of words), default: 0.2
keep_lastname_only = False # keep only the lastname? (supposed to be the first word) - this can enhance the matching if there are too many false positives, particularly if one database only include the last name but the other one has the full name

# Additional options
rename_columns_per_csv = ['', 'reports.'] # rename each column by prepending the csv file from where the column comes from. True will use the first 3 letters from the filename, or a list of 2 string prefixes can be provided, or False to disable renaming
#rename_columns_per_csv = False
pdmerge_indicator = False  # pandas.merge() option, if True, a new column 'x_merge' will be created for every columns, summarizing from what original database the info was merged from
pdmerge_validate = None  # pandas.merge() option, allows to ensure that the mapping is unique, for example with '1:1'. Can be: '1:1', '1:m', 'm:1', 'm:m'
blankna = False  # blank null values when saving to csv? This improves readability but the dates and numbers can't be reformatted then.
columns_order_first = ["name","name_alt6_cleanup","MRI date (on mri paper) + Dicom Date Sync With CRS-R + Date of RMN + date_assess + StudyDate","StudyDate","MRI date (on mri paper) + Dicom Date Sync With CRS-R","MRI date (on mri paper)","fmpagg.CRSr::Date and subscores","sarah.crs_subscore","fmpagg.CRSr::Best Computed Outcome","fmpagg.CRSr::Best Diagnosis","fmpagg.Final diagnosis","fmpagg.Dicom Diagnosis","manon.Diagnostic CRS-R at date of RMN","reports.final_diagnosis","reports.best_clinical_paraclinical_diagnosis","reports.diagnoses_clinical","steph.CRS-R on same day","manon.Diagnostic Final Comportemental", "manon.Diagnostic Final Para-clinique", "steph.MRI sedation","manon.sedation epi ","manon.sedation struct","fmpagg.MRI::Sedation EPI","fmpagg.MRI::Sedation Given","fmpagg.MRI::Sedation Structural","fmpagg.Dicom Path Sedation","reports.mri_sedation","fmpagg.AcuteDicom","""steph.acute in comments means that document was in folder "acute", but it might not be at the time of MRI/PET""","reports.acute","steph.gender","fmpagg.Sexe","reports.gender","manon.Age","reports.age","fmpagg.Etiology","fmpagg.Etiology specified.","manon.Aetiology","reports.accident_etiology","steph.etio (from PET, still need to look in acute folder and whole dossier_hospi folders and hospital anamnese on omnipro)","fmpagg.Date of Birth","manon.DB","reports.birthdate","fmpagg.Date of Accident","manon.Date of Injury","dicom.ProtocolName","dicom.SeriesDescription","dicom.id","dicom.match_{'ProtocolName': ['dti', 'repos']}","dicom.match_{'ProtocolName': ['repos']}","dicom.path","ecg.ecg_infos","ecg.ecg_types","reports.epileptic","steph.height","steph.weight","steph.comments MRI or PET","fmpagg.Cause of Death","fmpagg.Date of Death","fmpagg.DiagnosisOutcome","fmpagg.Diagnosis2Outcome","fmpagg.Diagnosis3Outcome","fmpagg.Diagnosis4Outcome","fmpagg.DateOutcome","fmpagg.Date2Outcome","fmpagg.Date3Outcome","fmpagg.Date4Outcome","fmpagg.GOSEOutcome","fmpagg.GOSE2Outcome","fmpagg.GOSE3Outcome","fmpagg.GOSE4Outcome","fmpagg.Days random CRSr since accident","fmpagg.Days scan since accident","fmpagg.Diabetes","fmpagg.Epilepsy","fmpagg.Handedness","fmpagg.PET::Arousal after injection","fmpagg.PET::Ctac / Elac","fmpagg.PET::Date of Assessment","fmpagg.PET::Sedation","fmpagg.PET::Weight Patient","fmpagg.SeriesDescription","fmpagg.ProtocolName","fmpagg.Remarks","fmpagg.deathdayssinceaccident","fmpagg.deathdayssinceinsult","manon.Comments","manon.Date CRS-R","manon.Diagnosis on admission","manon.PET","manon.Sedation + dose","manon.rest","manon.spatial","manon.svs","manon.tennis","reports.accident_date","reports.acquisition_date","reports.acquisition_date_end","reports.acquisition_minus_accident_days","reports.admission_diagnosis","reports.atypical_pattern","reports.diagnoses_fmri","reports.diagnoses_pet","reports.had_mri","reports.had_pet","reports.mental_disorders","reports.name_alt","reports.nociception_report","reports.ophtalmologic_report","reports.report_date","reports.report_lang","reports.report_path","reports.zolpidem_antirespondent","reports.zolpidem_context","reports.zolpidem_mention","reports.zolpidem_respondent","reports.zolpidem_tested","steph.CRS-R BEST (unreliable, from dossier_hospi summary sheet, might miss some CRS-Rs, I noticed that myself)","steph.CRS-R on day of PET scan","steph.PET date (on PET paper)","steph.PET sedation", ""]  # put these columns first. If None, no reordering will be done (except that the key columns will be first).

# Multi-columns merging
# keys to use for the merge: by default will only use the 'name' column, but can merge on multiple columns and with different types: 'id' or 'datetime' are supported
# the order of the columns must be the same, since we will not use the column names: the id's database first column must match the same type as the ref's database first column, etc for all subsequent columns
# format is: an OrderedDict with the column_name: column_type for each item
id_db_keys = OrderedDict([('name', 'id'), ('MRI date (on mri paper) + Dicom Date Sync With CRS-R + Date of RMN + date_assess + StudyDate', 'datetime|%Y-%m-%d')])
ref_db_keys = OrderedDict([('name', 'id')])
shared_key_column = 'name'  # in case you set ref_db_keys and id_db_keys to None (to disable multi-columns merging), you can specify here the name of the single column that both databases share, and on which the merge will be done
fillna = True  # if merging on multiple key columns, if fillna=True this will try to fill nan fields from other filled fields from the same subject, eg: subject A has 2 sessions, one with lots of infos, second session mostly empty, with fillna=True the second session will get all missing infos copied over from the first (except for the key columns of course)
fillna_exclude = ['MRI date (on mri paper)', 'Dicom Date Sync With CRS-R', 'Date of RMN', 'date_assess', 'StudyDate']  # allow to exclude some columns from being filled when fillna=True, this is useful particularly for datetime columns
join_on_shared_keys = True  # allow the merge on multi-columns to join on only the columns that are shared, this is useful when you want to groupby differently between dataframes (eg, first dataframe with name and date, the second with name only) but you want to keep all rows (else if you join on only name for example, all date rows will be concatenated)


In [ ]:
import pandas as pd

# Load first database
cid = pd.read_csv(id_db, sep=';', low_memory=False).dropna(how='all')
if id_db_tounicode:  # convert to unicode if required (can fix issues with accentuated characters)
    cid = df_to_unicode_fast(cid, progress_bar=True)
if id_db_keys is not None:
    print('IMPORTANT: Please check the date column (or any key column you used in your multikey) has the format you are expecting (because Excel does not show the true underlying format of some types, particularly dates).')
cid

In [ ]:
# Load second database
# Note that the convention "reference database" has no meaning anymore, it comes from an old version of this notebook where both databases had a different role, now it is not the case, only the id database is used as a reference for the names
cref = pd.read_csv(ref_db, sep=';', low_memory=False).dropna(how='all')
if ref_db_tounicode:  # convert to unicode if required (can fix issues with accentuated characters)
    cref = df_to_unicode_fast(cref, progress_bar=True)
if ref_db_keys is not None:
    print('IMPORTANT: Please check the date column (or any key column you used in your multikey) has the format you are expecting (because Excel does not show the true underlying format of some types, particularly dates).')
cref

In [ ]:
# Prepare prefix for columns renaming if rane_columns_per_csv is enabled
# We use the first 3 characters of each path plus a dot
# The user can also specify its own list of prefixes
if rename_columns_per_csv is True:
    prependcols = [os.path.basename(os.path.normpath(db))[:3]+'.' for db in [id_db, ref_db]]
elif isinstance(rename_columns_per_csv, list) and len(rename_columns_per_csv) == 2:
    prependcols = rename_columns_per_csv
else:
    prependcols = None
prependcols

In [ ]:
# Prepare key columns if using multiple columns as keys for the merge
if id_db_keys and ref_db_keys:
    keycol = [id_db_keys, ref_db_keys]
else:
    keycol = shared_key_column
keycol

In [ ]:
# Merge both databases if name matches (here we extract the names/indices where they match)
cmerge, cfinal = merge_two_df(cid, cref, col=keycol, mode=0,
                              dist_threshold=dist_threshold,
                              dist_words_threshold=dist_words_threshold,
                              skip_sanity=True, keep_nulls=True,
                              returnmerged=True, prependcols=prependcols,
                              fillna=fillna, fillna_exclude=fillna_exclude,
                              join_on_shared_keys=join_on_shared_keys,
                              indicator=pdmerge_indicator, validate=pdmerge_validate)
print('Done.')

In [ ]:
# Show the merge mapping
cmerge

In [ ]:
# Show the merged result (unified database)
cfinal

In [ ]:
# Reorder so that the first columns are always the key columns
if isinstance(keycol, list):
    # Deduplicate the key columns list (to avoid duplicating columns!)
    keycol_unique = []
    for kcol in keycol:
        for colname, coltype in kcol.items():
            if colname not in keycol_unique:
                keycol_unique.append(colname)
    # Reorder
    cfinal = reorder_cols_df(cfinal, keycol_unique)
else:
    # Only one key column, reorder according to it
    cfinal = reorder_cols_df(cfinal, [keycol])
cfinal

In [ ]:
# Save the merge mapping and unified database as csv files
cfinal_unicode = df_to_unicode_fast(cfinal)
cmerge_unicode = df_to_unicode_fast(cmerge)
if save_df_as_csv(cfinal_unicode, out_db, fields_order=columns_order_first if columns_order_first is not None else False, csv_order_by='name', blankna=blankna, date_format='%Y-%m-%d') and \
    save_df_as_csv(cmerge_unicode, out_db[:-4]+'_mapping.csv', fields_order=columns_order_first if columns_order_first is not None else False, csv_order_by='name', blankna=blankna):
    print('Merged database successfully saved in %s and %s!' % (out_db, out_db[:-4]+'_mapping.csv'))
else:
    print('ERROR: the merged database could not be saved!')